In [50]:
data = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""
data = open('puzzle.data').read()

def parse(data: str) -> tuple[set[complex], set[complex]]:
    rounds, cubes = set(), set()
    for y, line in enumerate(data.splitlines()):
        for x, c in enumerate(line):
            if c == '#':
                cubes.add(complex(x, y))
            elif c == 'O':
                rounds.add(complex(x, y))
    return tuple(rounds), tuple(cubes)

def tilt(rounds, cubes):
    new_set = set()
    for rock in sorted(rounds, key=lambda p: p.imag):
        y = 0
        for y in range(int(rock.imag), -1, -1):
            if complex(rock.real, y) in new_set or complex(rock.real, y) in cubes:
                new_set.add(complex(rock.real, y + 1))
                break
        else:
            new_set.add(complex(rock.real, 0))
    return new_set

def calc_load(rounds, cubes) -> int:
    rows = max(p.imag for p in rounds | set(cubes))
    load = [rows - p.imag + 1 for p in rounds]
    return int(sum(load))

def solve1(data: str):
    rounds, cubes = parse(data)
    rounds = tilt(rounds, cubes)
    return calc_load(rounds, cubes)

solve1(data)

109654

In [51]:
def rotate(rounds, cubes):
    columns = int(max(p.real for p in rounds | cubes))
    rounds = set(complex(columns - p.imag, p.real) for p in rounds)
    cubes = set(complex(columns - p.imag, p.real) for p in cubes)
    return rounds, cubes

def dump(rounds, cubes):
    rounds, cubes = set(rounds), set(cubes)
    for y in range(int(max(p.imag for p in rounds | cubes) + 1)):
        for x in range(int(max(p.real for p in rounds | cubes) + 1)):
            if complex(x, y) in cubes:
                print('#', end='')
            elif complex(x, y) in rounds:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print('\n')

def cycle(rounds, cubes):
    for _ in range(4):
        rounds = tilt(rounds, cubes)
        rounds, cubes = rotate(rounds, cubes)
    return tuple(rounds)

def solve2(data: str):
    CYCLES = 1_000_000_000
    states = dict()

    rounds, cubes = parse(data)
    for i in range(CYCLES):
        rounds = cycle(set(rounds), set(cubes))
        if rounds in states:
            start = states[rounds]
            break
        states[rounds] = i

    print(f'Found cycle starting at {start} with size of {i - start}')
    
    for i in range((CYCLES - start) % (i - start) - 1):
        rounds = cycle(set(rounds), set(cubes))  

    rows = max(p.imag for p in rounds + cubes)
    load = [rows - p.imag + 1 for p in rounds]
    return int(sum(load))

solve2(data)

Found cycle starting at 84 with size of 27


94876